In [2]:
# Install the necessary Python libraries
!pip install openai-whisper transformers openai librosa pydub accelerate

# Install ffmpeg (Required for audio processing with Whisper/pydub)
!apt-get install -y ffmpeg


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 803.2/803.2 kB 10.9 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for openai-whisper: filename=openai_whisper-20250625-py3-none-any.whl size=803979 sha256=101a735d897a150a7c7c77f87a0b0f62410c6cd01d6291fcb88da81f9f3740ba
  Stored in directory: /root/.cache/pip/wheels/61/d2/20/09ec9bef734d126cba375b15898010b6cc28578d8afdde5869
Successfully built openai-whisper
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
ffmpeg is already the newest version (7:4.4.2-0ubuntu0.22.04.1).
0 upgraded, 0 newly installed, 0 to remove and 41 not upgraded.


In [3]:
import librosa
import numpy as np
import whisper
from transformers import pipeline
from openai import OpenAI
import getpass

# Paste your key in the box prompted after running this cell
api_key = getpass.getpass("Enter your OpenAI API Key: ")

client = OpenAI(api_key=api_key)

# Initialize clients
sentiment_classifier = pipeline("audio-classification", model="superb/wav2vec2-base-superb-er")

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json: 0.00B [00:00, ?B/s]

/usr/local/lib/python3.12/dist-packages/transformers/configuration_utils.py:335: UserWarning: Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 Transformers. Using `model.gradient_checkpointing_enable()` instead, or if you are using the `Trainer` API, pass `gradient_checkpointing=True` in your `TrainingArguments`.
  warnings.warn(


pytorch_model.bin:   0%|          | 0.00/378M [00:00<?, ?B/s]

preprocessor_config.json:   0%|          | 0.00/215 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/378M [00:00<?, ?B/s]

Device set to use cpu


In [4]:
def analyze_audio(audio_path):
    y, sr = librosa.load(audio_path)

    # 1. Pitch & Monotonicity (Standard Deviation of Pitch)
    pitches, magnitudes = librosa.piptrack(y=y, sr=sr)
    pitch_std = np.std(pitches[pitches > 0]) # Higher std dev = less monotone

    # 2. Pace & Pauses (Simple energy detection)
    non_silent_intervals = librosa.effects.split(y, top_db=20)
    silence_duration = (len(y) - sum([end-start for start, end in non_silent_intervals])) / sr

    # 3. Emotion Detection (using HuggingFace)
    # Note: Requires 16kHz audio for most models
    emotion_result = sentiment_classifier(audio_path, top_k=1)
    dominant_emotion = emotion_result[0]['label']

    # 4. Heuristic Confidence Score (0-100)
    # Simple logic: penalize for silence and nervousness
    base_score = 80
    if dominant_emotion in ['sad', 'fear']: base_score -= 20
    if pitch_std < 10: base_score -= 10 # Penalize robotic voice

    return {
        "pitch_variation": pitch_std,
        "silence_seconds": silence_duration,
        "dominant_emotion": dominant_emotion,
        "delivery_score": base_score
    }

In [8]:
def analyze_content(audio_path):
    # 1. Transcribe
    model = whisper.load_model("base")
    result = model.transcribe(audio_path)
    transcript = result["text"]

    # 2. LLM Business Evaluation
    system_prompt = """
    You are a venture capital analyst. Analyze the pitch transcript for:
    1. Problem Clarity
    2. Revenue Model
    3. Market Opportunity
    4. Competitive Advantage
    Output a JSON with scores (0-100) for each and a brief critique.
    """

    response = client.chat.completions.create(
        model="gpt-5-nano",
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": transcript}
        ]
    )

    return transcript, response.choices[0].message.content

In [9]:
def generate_shark_feedback(transcript, audio_metrics, content_analysis):

    personas = {
        "The Visionary": "Focus on the big picture, future potential, and innovation. Ignore small financial details.",
        "The Skeptic": "Look for holes in the plan. Doubt the valuation. Question the competition.",
        "The Finance Shark": "Care only about margins, customer acquisition costs, and path to profitability."
    }

    feedback_report = {}

    context = f"""
    Pitch Transcript: {transcript}
    Audio Delivery Analysis: The speaker was {audio_metrics['dominant_emotion']} with a delivery score of {audio_metrics['delivery_score']}.
    Business Analysis: {content_analysis}
    """

    for shark_name, shark_style in personas.items():
        prompt = f"You are {shark_name}. Your style is: {shark_style}. Give a 2-sentence critique and a final 'Invest/No Invest' decision based on the context provided."

        response = client.chat.completions.create(
            model="gpt-5-nano",
            messages=[
                {"role": "system", "content": prompt},
                {"role": "user", "content": context}
            ]
        )
        feedback_report[shark_name] = response.choices[0].message.content

    return feedback_report

In [11]:
# Main Execution
audio_file = "user_pitch2.mp3"

print("--- Pipeline 1: Analyzing Voice ---")
audio_data = analyze_audio(audio_file)
print(f"Detected Emotion: {audio_data['dominant_emotion']}")

print("--- Pipeline 2: Analyzing Content ---")
transcript, business_score = analyze_content(audio_file)

print("--- Pipeline 3: Shark Tank Panel ---")
final_feedback = generate_shark_feedback(transcript, audio_data, business_score)

# Display Output
for shark, feedback in final_feedback.items():
    print(f"\n🦈 {shark}: {feedback}")

--- Pipeline 1: Analyzing Voice ---
Detected Emotion: hap
--- Pipeline 2: Analyzing Content ---


/usr/local/lib/python3.12/dist-packages/whisper/transcribe.py:132: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")


--- Pipeline 3: Shark Tank Panel ---

🦈 The Visionary: The concept of New Story as a transparent, video-verified, donor-driven home-building platform represents a powerful vision for scalable, accountable humanitarian funding and community resilience. However, the pitch lacks quantified problem scope, a sustainable revenue model, and a defensible moat to scale beyond Haiti, risking long-term viability.

Invest/No Invest: No Invest

🦈 The Skeptic: - Critique (2 sentences): The model relies on ongoing philanthropy and a corporate sponsor to cover operations, with no sustainable revenue stream, unit economics, or clear path to scale beyond Haiti. While donor transparency and impact videos are appealing, there’s no quantified problem size, TAM, defensible moat, or scalable plan to protect against competition and donor fatigue.

- Invest/No Invest: No Invest. The plan lacks a durable funding model, measurable metrics, and a scalable go-to-market/expansion strategy, making long-term viabilit